### 编译合约

In [1]:
//读取sol文件
var fs = require('fs');
var data = fs.readFileSync('add.sol','UTF-8'); //添加‘UTF-8’之后，得到字符串
data

'pragma solidity ^0.5.0;\n\ncontract Calc{\n\n  uint result;\n\n\n ' +
  ' function add(uint a, uint b) public returns(uint){\n    ' +
  'result = a + b;\n    return result;\n  }\n\n\n  function ' +
  'getCount() public view returns (uint){\n    return result;\n ' +
  ' }\n}'

In [2]:
var input = {
    language: 'Solidity',
    sources: {
        'add.sol' : {
            content: data
        }
    },
    settings: {
        outputSelection: {
            '*': {
                '*': [ '*' ]
            }
        }
    }
};

In [3]:
//引入web3 
var web3 = require('web3')
// 使用eth之前，需要绑定
var web3 = new web3('HTTP://127.0.0.1:7545')

In [4]:
var list;
web3.eth.getAccounts().then(function(add){
    list = add
})

In [5]:
// 合约部署者的地址 
var deployer = list[0];
deployer

'0x53a2DaAe7B69138fF57cBBc2F94866Bf72f8b9BF'

In [6]:
// web3.eth.compile.solidity() 失效
var solc = require('solc')

In [7]:
var output = JSON.parse(solc.compile(JSON.stringify(input)));
output

{
  contracts: { 'add.sol': { Calc: [Object] } },
  sources: { 'add.sol': { id: 0 } }
}

In [14]:
for (var contractName in output.contracts['add.sol']) {
	console.log(contractName + ': ' + output.contracts['add.sol'][contractName].evm.bytecode.object)
}

Calc: 608060405234801561001057600080fd5b5060f08061001f6000396000f3fe6080604052348015600f57600080fd5b506004361060325760003560e01c8063771602f7146037578063a87d942c146080575b600080fd5b606a60048036036040811015604b57600080fd5b810190808035906020019092919080359060200190929190505050609c565b6040518082815260200191505060405180910390f35b608660b2565b6040518082815260200191505060405180910390f35b6000818301600081905550600054905092915050565b6000805490509056fea265627a7a723158208db16baf38ecc1c01a9acbd39dd828a73f3541e02215320a3630527772f6957864736f6c634300050b0032


In [15]:
// 一定要使用'0x'+ object
var bytecode = output.contracts['add.sol']['Calc'].evm.bytecode.object
bytecode

'608060405234801561001057600080fd5b5060f08061001f6000396000f3fe6080604052348015600f57600080fd5b506004361060325760003560e01c8063771602f7146037578063a87d942c146080575b600080fd5b606a60048036036040811015604b57600080fd5b810190808035906020019092919080359060200190929190505050609c565b6040518082815260200191505060405180910390f35b608660b2565b6040518082815260200191505060405180910390f35b6000818301600081905550600054905092915050565b6000805490509056fea265627a7a723158208db16baf38ecc1c01a9acbd39dd828a73f3541e02215320a3630527772f6957864736f6c634300050b0032'

In [28]:
web3.utils.toHex(bytecode)

'0x3630383036303430353233343830313536313030313035373630303038306664356235303630663038303631303031663630303033393630303066336665363038303630343035323334383031353630306635373630303038306664356235303630303433363130363033323537363030303335363065303163383036333737313630326637313436303337353738303633613837643934326331343630383035373562363030303830666435623630366136303034383033363033363034303831313031353630346235373630303038306664356238313031393038303830333539303630323030313930393239313930383033353930363032303031393039323931393035303530353036303963353635623630343035313830383238313532363032303031393135303530363034303531383039313033393066333562363038363630623235363562363034303531383038323831353236303230303139313530353036303430353138303931303339306633356236303030383138333031363030303831393035353530363030303534393035303932393135303530353635623630303038303534393035303930353666656132363536323761376137323331353832303864623136626166333865636331633031613961636264333964643832386137336633353431653032323

In [20]:
var abi = output.contracts['add.sol']['Calc'].abi
abi 

[
  {
    constant: false,
    inputs: [ [Object], [Object] ],
    name: 'add',
    outputs: [ [Object] ],
    payable: false,
    stateMutability: 'nonpayable',
    type: 'function',
    signature: '0x771602f7'
  },
  {
    constant: true,
    inputs: [],
    name: 'getCount',
    outputs: [ [Object] ],
    payable: false,
    stateMutability: 'view',
    type: 'function',
    signature: '0xa87d942c'
  }
]

### 部署合约

In [29]:
// 创建合约对象
var calcContract = new web3.eth.Contract(abi,null,{
    data:'0x' + bytecode,
    gas:6000000
})

In [30]:
// 部署合约
calcContract.deploy().send({
    from: deployer
})
.on('error', (error) => { 
    console.error(error)
 })
.on('transactionHash', (transactionHash) => { 
    console.log("transactionHash :" + transactionHash)
 })
.on('receipt', (receipt) => {
   console.log("receipt:") 
   console.log(receipt) 
})
.on('confirmation', (confirmationNumber, receipt) => { 
   console.log("confirmationNumber:"+confirmationNumber)
 }).then((newContractInstance) => {
    console.log(newContractInstance)
    console.log(newContractInstance.options.address) // instance with the new contract address
});

transactionHash :0xcd648843cf37a3439aafabe3b759d43c0069e5bfbe4afc40829eb070692b1339
receipt:
{
  transactionHash: '0xcd648843cf37a3439aafabe3b759d43c0069e5bfbe4afc40829eb070692b1339',
  transactionIndex: 0,
  blockHash: '0xd9b2386825d448910dfe81de60cfce3dbf0c14695826b6301656c75a701926f7',
  blockNumber: 10,
  from: '0x53a2daae7b69138ff57cbbc2f94866bf72f8b9bf',
  to: null,
  gasUsed: 118567,
  cumulativeGasUsed: 118567,
  contractAddress: '0x04cEEb858aDCf2724cd7BFaAcC7BFaC02b002482',
  status: true,
  logsBloom: '0x000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

### 调用合约

In [35]:
// 获得合约实例
var contractInstance =new web3.eth.Contract(abi, '0x04cEEb858aDCf2724cd7BFaAcC7BFaC02b002482')
contractInstance

Contract {
  currentProvider: [Getter/Setter],
  _requestManager: RequestManager {
    provider: HttpProvider {
      host: 'HTTP://127.0.0.1:7545',
      httpAgent: [Agent],
      timeout: 0,
      headers: undefined,
      connected: true
    },
    providers: {
      WebsocketProvider: [Function: WebsocketProvider],
      HttpProvider: [Function: HttpProvider],
      IpcProvider: [Function: IpcProvider]
    },
    subscriptions: {}
  },
  givenProvider: null,
  providers: {
    WebsocketProvider: [Function: WebsocketProvider],
    HttpProvider: [Function: HttpProvider],
    IpcProvider: [Function: IpcProvider]
  },
  _provider: HttpProvider {
    host: 'HTTP://127.0.0.1:7545',
    httpAgent: Agent {
      _events: [Object],
      _eventsCount: 1,
      _maxListeners: undefined,
      defaultPort: 80,
      protocol: 'http:',
      options: [Object],
      requests: {},
      sockets: {},
      freeSockets: {},
      keepAliveMsecs: 1000,
      keepAlive: true,
      maxSockets: Infi

In [37]:
//without sending any transaction.
contractInstance.methods.add(4,5).call({
    from:deployer
})

'9'

In [40]:
contractInstance.methods.getCount().call({
    from:deployer
})

'0'

In [41]:
// Will send a transaction to the smart contract and execute its method. Note this can alter the smart contract state.
contractInstance.methods.add(8,9).send({
    from:deployer
})

{
  transactionHash: '0x8b150cd102f7d90efaee66628c3d951316665a010447603a85677cd7c34757a9',
  transactionIndex: 0,
  blockHash: '0xd9affe92c3d51fca529c70c9c64840fad9f35f685e742945b134add4f6acd6ad',
  blockNumber: 11,
  from: '0x53a2daae7b69138ff57cbbc2f94866bf72f8b9bf',
  to: '0x04ceeb858adcf2724cd7bfaacc7bfac02b002482',
  gasUsed: 42183,
  cumulativeGasUsed: 42183,
  contractAddress: null,
  status: true,
  logsBloom: '0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000',
  v: '0x1b',
  r: '0xba85f1d1d2d97d9190d5b78815457a19b79ffce

In [42]:
contractInstance.methods.getCount().call()

'17'

In [43]:
contractInstance.methods.getCount().call()
.then(console.log);

17


In [44]:
contractInstance.methods.getCount().call()

'17'